<a href="https://colab.research.google.com/github/choiy4432/AICV03_Missing_line_segmentation_ensemble_model/blob/main/00_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################################################
# 🔥 LOID-Cityscapes 00_setup
######################################################################

# 1. 기존 삭제 + 새 PyTorch (Colab 최적)
!pip install torch torchvision torchaudio

print("\n🔗 GitHub: https://github.com/choiy4432/AICV03_Missing_line_segmentation_ensemble_model")


In [ ]:
# 2. 나머지 패키지 일괄 설치
!pip install ultralytics
!pip install simple-lama-inpainting segmentation-models-pytorch
!pip install cityscapesscripts
!pip install opencv-python
!pip install albumentations
!pip install matplotlib

In [ ]:
# 3. 검증
import torch
print(f"✅ PyTorch {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()} | {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

In [ ]:
# 4. 핵심 import
from ultralytics import YOLO
import segmentation_models_pytorch as smp
from simple_lama_inpainting import SimpleLama
print("✅ YOLO/DeepLab/LaMa import 성공!")

In [ ]:
# 5. Day1 핵심: seg_baseline 함수 + 테스트
import numpy as np, cv2
class LaneSegModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = smp.DeepLabV3Plus("resnet34", classes=1, activation=None)
    def forward(self, x):
        return torch.sigmoid(self.model(x)).squeeze(1)

model = LaneSegModel()
if torch.cuda.is_available(): model = model.cuda()

@torch.no_grad()
def seg_baseline(image_np, model, device="cuda"):
    """Day1 목표! image[H,W,3] → prob[H,W]"""
    model.eval()
    img = cv2.resize(image_np, (1024,512))
    x = torch.FloatTensor(img.transpose(2,0,1)).unsqueeze(0).to(device)/255.
    prob = model(x).squeeze().cpu().numpy()
    return cv2.resize(prob, (image_np.shape[1], image_np.shape[0]))

In [ ]:
# 테스트
test_img = np.random.randint(100,200,(512,1024,3), np.uint8)
P = seg_baseline(test_img, model)
print(f"✅ seg_baseline 완성: {P.shape} ({P.min():.2f}~{P.max():.2f})")

In [ ]:
# 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.subplot(131); plt.imshow(test_img); plt.title("Input")
plt.subplot(132); plt.imshow(P, cmap="jet", vmin=0, vmax=1); plt.title("seg_baseline")
plt.subplot(133); plt.hist(P.ravel(), bins=50, color='orange'); plt.title("Prob Dist")
plt.tight_layout()
plt.savefig("day1_test.png", dpi=150, bbox_inches='tight')
plt.show()